# Taller de Redes Neuronales Convolucionales



## ¿Por qué CIFAR-10 es ideal para redes convolucionales?

Las imágenes tienen características que las convoluciones aprovechan muy bien:

- **Estructura espacial**: Los píxeles cercanos están relacionados entre sí (un ojo está cerca de otro ojo, las ruedas están debajo del coche, etc.)
- **Patrones locales**: Bordes, texturas y formas son características que aparecen en regiones pequeñas de la imagen
- **Invarianza a la traslación**: Un gato sigue siendo un gato esté en la esquina o en el centro de la imagen
- **Tamaño manejable**: El dataset cabe en memoria sin problemas, perfecto para experimentar

## 1. Cargar el Dataset CIFAR-10

CIFAR-10 es un dataset clásico en visión por computadora. Contiene 60,000 imágenes a color de 32×32 píxeles distribuidas en 10 categorías:

**Las 10 clases son:** avión, automóvil, pájaro, gato, ciervo, perro, rana, caballo, barco y camión.



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import cifar10

# Cargamos el dataset (se descarga automáticamente si es la primera vez)
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Nombres de las clases en español
nombres_clases = ['avión', 'automóvil', 'pájaro', 'gato', 'ciervo', 
                  'perro', 'rana', 'caballo', 'barco', 'camión']

print("Forma del conjunto de entrenamiento:", x_train.shape)
print("Forma de las etiquetas de entrenamiento:", y_train.shape)
print("Forma del conjunto de prueba:", x_test.shape)
print("Forma de las etiquetas de prueba:", y_test.shape)

## 2. Análisis Exploratorio de Datos (EDA)

Antes de entrenar cualquier modelo, es fundamental entender lo que se esta trabajando. Vamos a explorar la estructura del dataset, ver ejemplos de cada clase y analizar la distribución de los datos.

### 2.1 Estructura general del dataset

In [ ]:
# Veamos las características principales del dataset
print("=" * 50)
print("ESTRUCTURA DEL DATASET")
print("=" * 50)
print(f"Imágenes de entrenamiento: {x_train.shape[0]:,}")
print(f"Imágenes de prueba: {x_test.shape[0]:,}")
print(f"Dimensiones de cada imagen: {x_train.shape[1]} × {x_train.shape[2]} píxeles")
print(f"Canales de color: {x_train.shape[3]} (RGB)")
print(f"Rango de valores de píxel: [{x_train.min()}, {x_train.max()}]")
print(f"Tipo de dato: {x_train.dtype}")
print(f"Total de características por imagen: {32 * 32 * 3:,} valores")

### 2.2 Distribución de clases

Un aspecto importante es verificar si el dataset está balanceado. Si una clase tiene muchas más muestras que otra, el modelo podría sesgarse hacia la clase mayoritaria.

In [ ]:
# Contamos cuántas imágenes hay de cada clase
valores_unicos, conteos = np.unique(y_train, return_counts=True)
distribucion = dict(zip([nombres_clases[i] for i in valores_unicos], conteos))

print("Distribución de clases en el conjunto de entrenamiento:\n")
for clase, cantidad in distribucion.items():
    porcentaje = cantidad / len(y_train) * 100
    print(f"  {clase:12s}: {cantidad:,} imágenes ({porcentaje:.1f}%)")

# Visualizamos la distribución
plt.figure(figsize=(10, 4))
plt.bar([nombres_clases[i] for i in valores_unicos], conteos, color='steelblue')
plt.title('Distribución de Clases en CIFAR-10')
plt.xlabel('Clase')
plt.ylabel('Cantidad de imágenes')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print("\n✓ El dataset está perfectamente balanceado: 5,000 imágenes por clase")

### 2.3 Visualización de ejemplos

Nada mejor que ver las imágenes para entender con qué estamos trabajando. Primero veamos un ejemplo de cada clase:

In [ ]:
# Mostramos una imagen de cada clase
fig, axes = plt.subplots(2, 5, figsize=(12, 5))
axes = axes.flatten()

for idx_clase in range(10):
    # Buscamos la primera imagen de esta clase
    idx_imagen = np.where(y_train.flatten() == idx_clase)[0][0]
    axes[idx_clase].imshow(x_train[idx_imagen])
    axes[idx_clase].set_title(nombres_clases[idx_clase])
    axes[idx_clase].axis('off')

plt.suptitle('Un ejemplo de cada clase', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Veamos más variedad: 5 ejemplos de cada clase
fig, axes = plt.subplots(10, 5, figsize=(10, 18))

for idx_clase in range(10):
    # Obtenemos los índices de las primeras 5 imágenes de esta clase
    indices = np.where(y_train.flatten() == idx_clase)[0][:5]
    for j, idx in enumerate(indices):
        axes[idx_clase, j].imshow(x_train[idx])
        if j == 0:
            axes[idx_clase, j].set_ylabel(nombres_clases[idx_clase], fontsize=10)
        axes[idx_clase, j].axis('off')

plt.suptitle('Variedad de ejemplos por clase', fontsize=14)
plt.tight_layout()
plt.show()

print("Observa cómo hay mucha variación dentro de cada clase:")
print("- Diferentes fondos, ángulos, iluminación y tamaños")
print("- Esto hace que la clasificación sea un reto interesante")

### 2.4 Distribución de valores de píxel

También es útil ver cómo se distribuyen los valores de intensidad en cada canal de color (Rojo, Verde, Azul).

In [ ]:
# Histogramas de cada canal de color
fig, axes = plt.subplots(1, 3, figsize=(12, 3))

canales = ['Rojo', 'Verde', 'Azul']
colores = ['red', 'green', 'blue']

for i, (nombre, color) in enumerate(zip(canales, colores)):
    axes[i].hist(x_train[:, :, :, i].flatten(), bins=50, color=color, alpha=0.7)
    axes[i].set_title(f'Canal {nombre}')
    axes[i].set_xlabel('Valor del píxel')
    axes[i].set_ylabel('Frecuencia')

plt.suptitle('Distribución de valores de píxel por canal (antes de normalizar)', fontsize=12)
plt.tight_layout()
plt.show()

## 3. Preprocesamiento de Datos

Antes de entrenar el modelo, necesitamos preparar los datos. Esto incluye dos pasos fundamentales:

### 3.1 Normalización de píxeles

Las imágenes originales tienen valores entre 0 y 255. Las redes neuronales funcionan mejor cuando los valores de entrada son pequeños (entre 0 y 1). La solución es simple: dividimos cada valor por 255.

$$
x_{normalizado} = \frac{x_{original}}{255}
$$

**¿Por qué normalizamos?**
- Los gradientes se mantienen en rangos razonables
- El entrenamiento es más estable y rápido
- Evitamos problemas numéricos

In [ ]:
# Normalizamos los valores de píxel al rango [0, 1]
x_train_norm = x_train.astype('float32') / 255.0
x_test_norm = x_test.astype('float32') / 255.0

print(f"Rango de píxeles después de normalizar: [{x_train_norm.min()}, {x_train_norm.max()}]")
print(f"Tipo de dato: {x_train_norm.dtype}")

### 3.2 Codificación One-Hot de las etiquetas

Las etiquetas originales son números enteros del 0 al 9. Sin embargo, nuestra red neuronal usará una capa softmax que produce 10 probabilidades (una por clase). Para calcular la pérdida, necesitamos que las etiquetas también sean vectores de 10 elementos.

**Ejemplo de codificación one-hot:**

| Etiqueta original | Clase | Vector one-hot |
|-------------------|-------|----------------|
| 3 | gato | [0, 0, 0, **1**, 0, 0, 0, 0, 0, 0] |
| 0 | avión | [**1**, 0, 0, 0, 0, 0, 0, 0, 0, 0] |
| 7 | caballo | [0, 0, 0, 0, 0, 0, 0, **1**, 0, 0] |

In [ ]:
from tensorflow.keras.utils import to_categorical

# Convertimos las etiquetas a formato one-hot
y_train_oh = to_categorical(y_train, num_classes=10)
y_test_oh = to_categorical(y_test, num_classes=10)

print(f"Forma original de etiquetas: {y_train.shape}")
print(f"Forma después de one-hot: {y_train_oh.shape}")

# Veamos un ejemplo concreto
etiqueta_original = y_train[0][0]
print(f"\nEjemplo:")
print(f"  Etiqueta original: {etiqueta_original} ({nombres_clases[etiqueta_original]})")
print(f"  Vector one-hot: {y_train_oh[0]}")

### 3.3 Verificación del preprocesamiento

Siempre es buena idea verificar que todo esté correcto antes de continuar:

In [ ]:
# Mostramos algunas imágenes normalizadas para verificar que se ven bien
fig, axes = plt.subplots(1, 5, figsize=(10, 2))

for i in range(5):
    axes[i].imshow(x_train_norm[i])
    axes[i].set_title(nombres_clases[y_train[i][0]])
    axes[i].axis('off')

plt.suptitle('Imágenes normalizadas (valores entre 0 y 1)', fontsize=12)
plt.tight_layout()
plt.show()

print("\n" + "=" * 50)
print("✓ PREPROCESAMIENTO COMPLETADO")
print("=" * 50)
print(f"  Datos de entrenamiento: {x_train_norm.shape}")
print(f"  Etiquetas de entrenamiento: {y_train_oh.shape}")
print(f"  Datos de prueba: {x_test_norm.shape}")
print(f"  Etiquetas de prueba: {y_test_oh.shape}")
print("\n¡Los datos están listos para entrenar!")